# Knotworking: An Inverse-Problem Approach to Phase I Drug Discovery

## Project Description

This project implements a **10-layer end-to-end computational drug discovery pipeline** that reframes early-stage drug discovery as an *inverse problem*: rather than screening existing compound libraries, we start from a patient's genomic and clinical profile and work *backwards* to generate and validate novel candidate molecules that are mathematically proven to satisfy drug-likeness constraints.

The pipeline is novel in three ways:

1. **Patient-first reasoning** — Layers 1–6 translate raw patient data (symptoms, genetic variants, lab values) into a ranked, high-confidence drug target before any chemistry is done.
2. **Generative molecular design** — Layer 7 uses a Bayesian Graph Variational Autoencoder (BayesianGraphVAE) trained on 250 k ZINC drug-like molecules to *generate* candidate structures conditioned on the identified target, guided by a feedback loop from the validation layer.
3. **Formal verification** — Layer 9 uses the Rocq (Coq) proof assistant together with an LLM (Claude) to produce machine-checkable proofs that each candidate molecule satisfies Lipinski's Rule of Five, Veber rules, and structural safety constraints — providing regulatory-grade auditability that black-box ML alone cannot.

### Key Technologies
| Component | Technology |
|-----------|------------|
| Patient genomics | Ensembl VEP, gnomAD, ClinVar APIs |
| Protein interaction graph | STRING DB, Reactome |
| Target scoring | Open Targets Platform, ChEMBL |
| Molecular generation | BayesianGraphVAE (PyTorch) |
| Property prediction | MC-Dropout MLP — 6 outputs |
| Formal verification | Rocq/Coq (14 chemistry axiom files) |
| LLM proof generation | Claude 3.5 Haiku (Anthropic) |
| Cheminformatics | RDKit |

---

## Architecture Completeness Review

The README specifies a 10-layer pipeline. The table below maps each layer to its implementation status.

| Layer | README Description | Implementation File(s) | Status |
|-------|--------------------|------------------------|--------|
| 1 | Data Ingestion (patient data → standardised format) | `src/data/data_loader.py` (ZINC CSV only) | ⚠ Partial — clinical/imaging ingestion missing |
| 2 | Disease Prediction ML (diagnose subtype/severity) | `src/pipeline/layer2_patient_intake.py` | ⚠ Partial — schema + validation done; ML classifier not implemented |
| 3 | Genotype-to-Phenotype (GWAS, variant effect) | `src/pipeline/layer3_genotype_phenotype.py` | ✅ Complete |
| 4 | Causal Pathway Modeling (protein interaction graph) | `src/pipeline/layer4_causal_pathway.py` | ✅ Complete |
| 5 | Target Identification (druggable proteins) | `src/pipeline/layer5_target_identification.py` | ✅ Complete |
| 6 | Target Confidence Scoring (multi-signal aggregation) | `src/pipeline/layer6_target_confidence.py` | ✅ Complete |
| 7 | Molecular Generation (VAE/diffusion/RL) | `model.py`, `run_pipeline.py` | ⚠ Partial — VAE architecture done; generation uses hardcoded SMILES pool |
| 8 | Property Prediction (binding, solubility, toxicity) | `src/ml/property_predictor.py` | ✅ Complete |
| 9 | Computational Drug Validation (RDKit + formal proof) | `src/llm/pipeline_layer9.py`, `src/rocq/*.v` | ✅ Complete |
| 10 | Multi-Fidelity Screening (ML → MD → QM → experiments) | — | ❌ Not implemented |

**Feedback loop** (Layer 9 → Layer 7): ✅ Implemented in `src/llm/feedback_controller.py`

### Gap Summary
- **Layer 1**: No parser for medical imaging or EHR documents; only tabular CSV is handled.
- **Layer 2**: The README specifies an ML disease classifier; the current code only provides a structured data schema (`PatientRecord`) and input validation — no trained model.
- **Layer 7**: `BayesianGraphVAE` is fully defined and trains correctly, but `run_pipeline.py` samples from a hardcoded SMILES pool instead of decoding from the latent space of a trained model.
- **Layer 10**: Placeholder only — no molecular dynamics, quantum chemistry, or experimental integration exists.

**Overall completeness: ~70%** — all critical validation and target-selection logic is production-quality; the main gaps are generative model deployment (L7) and multi-fidelity screening (L10).

---
## Setup & Imports

In [ ]:
# Standard library
import os
import sys
import json
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
PROJECT_ROOT = os.path.abspath('.')
sys.path.insert(0, PROJECT_ROOT)

# Scientific stack
import numpy as np
import pandas as pd

# Deep learning
import torch
import torch.nn as nn

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors, QED, AllChem
from rdkit.Chem import rdMolDescriptors

print('Python     :', sys.version.split()[0])
print('PyTorch    :', torch.__version__)
print('NumPy      :', np.__version__)
print('Pandas     :', pd.__version__)
print('CUDA avail :', torch.cuda.is_available())

---
## Layer 1 — Data Ingestion

Load and inspect the ZINC 250k drug-like molecule dataset.  
**TODO**: extend to accept clinical JSON / HL7 FHIR patient records.

In [ ]:
from src.data.data_loader import prepare_dataset

# Prepare dataset: fingerprints, labels, toxic flags
# Returns tensors written to disk; loads them here for inspection
X, y, toxic = prepare_dataset()

print(f'Fingerprint matrix : {X.shape}   (N_molecules x 2048 bits)')
print(f'QED labels         : {y.shape}')
print(f'Toxic flags        : {toxic.shape}  ({toxic.sum().item()} flagged)')
print(f'Mean QED           : {y.mean():.3f}  (1.0 = perfectly drug-like)')

In [ ]:
# Quick sanity check — inspect a few molecules
df = pd.read_csv('src/data/250k_rndm_zinc_drugs_clean_3.csv').sample(5, random_state=0)
df[['smiles', 'qed']].head()

---
## Layer 2 — Patient Intake & Disease Prediction

`PatientRecord` captures demographics, clinical data, and genetic variants.  
**TODO**: train an ML classifier (e.g. gradient-boosted tree or transformer) on top of this schema to predict disease subtype/severity.

In [ ]:
from src.pipeline.layer2_patient_intake import PatientRecord, GeneVariant, LabValue

# Example patient record
patient = PatientRecord(
    patient_id='PT-0001',
    age=52,
    sex='M',
    symptoms=['fatigue', 'weight loss', 'night sweats'],
    diagnosis='Non-Hodgkin Lymphoma',
    disease_subtype='DLBCL',
    severity='stage-III',
    genes_of_interest=['MYC', 'BCL2', 'TP53'],
    gene_variants=[
        GeneVariant(gene='MYC', hgvs='c.202G>A', variant_type='missense', zygosity='heterozygous'),
        GeneVariant(gene='TP53', hgvs='c.817C>T', variant_type='nonsense', zygosity='homozygous'),
    ],
    lab_values=[
        LabValue(name='LDH', value=420.0, unit='U/L', reference_range=(120, 240)),
        LabValue(name='WBC', value=14.2,  unit='×10⁹/L', reference_range=(4.0, 11.0)),
    ],
)

patient.validate()
print('HGVS notations:', patient.hgvs_notations())
print('Serialised record:')
print(json.dumps(patient.to_dict(), indent=2))

---
## Layer 3 — Genotype-to-Phenotype Association

Calls Ensembl VEP, gnomAD, and ClinVar to annotate each variant and identify dysfunctional proteins.  
> **Note**: requires internet access and may take ~10–30 s per variant.

In [ ]:
from src.pipeline.layer3_genotype_phenotype import GenotypePhenotypeMapper

mapper = GenotypePhenotypeMapper()
gp_result = mapper.run(patient)

print('Dysfunctional proteins identified:')
for protein in gp_result.dysfunctional_proteins:
    print(f'  • {protein}')

print('\nVariant consequences:')
for vc in gp_result.variant_consequences:
    print(f'  {vc.hgvs}: impact={vc.impact}, consequence={vc.consequence}')

print('\nGene constraint scores (pLI > 0.9 → not safe to target):')
for gc in gp_result.gene_constraints:
    print(f'  {gc.gene}: pLI={gc.pli:.2f}, LOEUF={gc.loeuf:.2f}')

---
## Layer 4 — Causal Pathway Modeling

Builds a directed protein–protein interaction graph using STRING DB and Reactome, then ranks nodes by betweenness centrality (bottleneck = best target).

In [ ]:
from src.pipeline.layer4_causal_pathway import CausalPathwayModeler

modeler = CausalPathwayModeler()
pathway_result = modeler.run(gp_result.dysfunctional_proteins)

print('Top pathway candidates (by betweenness centrality):')
for i, candidate in enumerate(pathway_result.top_candidates[:5], 1):
    print(f'  {i}. {candidate.protein}  centrality={candidate.centrality:.4f}')

---
## Layer 5 — Target Identification

Queries the Open Targets Platform and ChEMBL to score each candidate protein on druggability, tractability, and known drug interactions.

In [ ]:
from src.pipeline.layer5_target_identification import TargetIdentifier

identifier = TargetIdentifier()
target_result = identifier.run(pathway_result.top_candidates, patient)

print(f'Chosen target: {target_result.chosen_target.gene}')
print(f'  Open Targets association score : {target_result.chosen_target.ot_score:.3f}')
print(f'  ChEMBL known drugs             : {target_result.chosen_target.chembl_drug_count}')
print(f'  Tractability (small molecule)  : {target_result.chosen_target.tractability_sm}')
print(f'  Binding pocket exists          : {target_result.chosen_target.has_binding_pocket}')

---
## Layer 6 — Target Confidence Scoring

Combines multi-signal evidence into a single deterministic confidence score [0, 1] for regulatory auditability. Essential genes and safety-flagged targets are penalised.

In [ ]:
from src.pipeline.layer6_target_confidence import TargetConfidenceScorer

scorer = TargetConfidenceScorer()
confidence = scorer.score(
    target_result.chosen_target,
    pathway_result,
    gp_result,
)

print(f'Target confidence score: {confidence.final_score:.3f}')
print(f'  OT contribution       : {confidence.ot_contribution:.3f}')
print(f'  Centrality bonus      : {confidence.centrality_contribution:.3f}')
print(f'  Essential gene penalty: {confidence.essential_penalty:.3f}')
print(f'  Safety flag penalty   : {confidence.safety_penalty:.3f}')

CONFIDENCE_THRESHOLD = 0.5
if confidence.final_score >= CONFIDENCE_THRESHOLD:
    print(f'\n✅ Target cleared for molecular generation (score ≥ {CONFIDENCE_THRESHOLD})')
else:
    print(f'\n❌ Target below threshold — revisit Layer 5 candidates')

---
## Layer 7 — Molecular Generation (BayesianGraphVAE)

Trains (or loads) the Bayesian Graph VAE on ZINC fingerprints, then decodes candidate molecules from the latent space.  
**TODO**: condition the latent-space sampling on the target protein embedding to bias generation towards binders.

In [ ]:
from model import BayesianGraphVAE

FINGERPRINT_DIM = 2048
LATENT_DIM      = 16
MODEL_PATH      = 'models/model.pt'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vae    = BayesianGraphVAE(input_dim=FINGERPRINT_DIM, latent_dim=LATENT_DIM).to(device)

if os.path.exists(MODEL_PATH):
    vae.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    print(f'Loaded trained VAE from {MODEL_PATH}')
else:
    print('No saved model found — training from scratch (300 epochs).')
    print('Run: python model.py   to train and save.')

In [ ]:
# --- Training loop (skip if model already saved) ---
TRAIN_VAE = not os.path.exists(MODEL_PATH)   # set True to force re-training

if TRAIN_VAE:
    from torch.utils.data import DataLoader, TensorDataset

    X_train = X.to(device)
    dataset = TensorDataset(X_train)
    loader  = DataLoader(dataset, batch_size=256, shuffle=True)

    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    EPOCHS    = 300

    vae.train()
    for epoch in range(1, EPOCHS + 1):
        total_loss = 0.0
        for (batch,) in loader:
            optimizer.zero_grad()
            loss, *_ = vae(batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if epoch % 50 == 0:
            print(f'Epoch {epoch:3d}/{EPOCHS}  loss={total_loss / len(loader):.4f}')

    os.makedirs('models', exist_ok=True)
    torch.save(vae.state_dict(), MODEL_PATH)
    print('Model saved to', MODEL_PATH)

In [ ]:
# Generate candidate molecules by sampling the latent space
vae.eval()
N_CANDIDATES = 8

with torch.no_grad():
    z         = torch.randn(N_CANDIDATES, LATENT_DIM, device=device)
    fp_recon  = vae.decode(z).cpu().numpy()        # (N, 2048) soft fingerprints
    fp_binary = (fp_recon > 0.5).astype(int)

print(f'Generated {N_CANDIDATES} candidate fingerprints.')
print('Mean bit density:', fp_binary.mean())

# NOTE: Converting binary fingerprints back to valid SMILES requires
# a fingerprint-inversion model or nearest-neighbour lookup in the training set.
# TODO: integrate REINVENT / graph-based decoder for valid SMILES output.

---
## Layer 8 — Property Prediction (MC-Dropout MLP)

Predicts 6 physicochemical properties with uncertainty estimates via Monte Carlo Dropout (50 forward passes).

In [ ]:
from src.ml.property_predictor import PropertyPredictor

PROPERTY_NAMES = ['QED', 'LogP', 'MW', 'TPSA', 'HBD', 'HBA']
predictor = PropertyPredictor(input_dim=FINGERPRINT_DIM, n_outputs=len(PROPERTY_NAMES))

# Predict on the first generated fingerprint using MC-Dropout
sample_fp = torch.tensor(fp_binary[:1], dtype=torch.float32)
means, variances = predictor.predict_with_uncertainty(sample_fp, n_samples=50)

print('Property predictions for candidate #0:')
print(f'{"Property":<8} {"Mean":>10} {"Std Dev":>10}')
print('-' * 32)
for name, mu, var in zip(PROPERTY_NAMES, means[0], variances[0]):
    print(f'{name:<8} {mu:>10.3f} {var**0.5:>10.3f}')

---
## Layer 9 — Computational Drug Validation (Rocq + LLM)

Four-checkpoint validation pipeline:
1. **Lipinski Rule of Five** — MW ≤ 500, LogP ≤ 5, HBD ≤ 5, HBA ≤ 10
2. **Veber Rules** — RotBonds ≤ 10, PSA ≤ 140
3. **Safety Filters** — PAINS & Brenk structural alerts (via RDKit)
4. **Formal Rocq Proof** — LLM-generated, machine-verified by `coqc`

> **Requires**: Anthropic API key set as `ANTHROPIC_API_KEY` in the environment, and `coqc` installed.

In [ ]:
from src.llm.pipeline_layer9 import DrugValidator

# Use an example SMILES (aspirin) for demonstration
DEMO_SMILES = 'CC(=O)Oc1ccccc1C(=O)O'   # Aspirin

validator = DrugValidator()
result    = validator.validate(DEMO_SMILES)

print(f'SMILES           : {DEMO_SMILES}')
print(f'Confidence score : {result.confidence_score:.2f}  ({result.confidence_score*100:.0f}% checks passed)')
print()
for check_name, passed in result.checks.items():
    icon = '✅' if passed else '❌'
    print(f'  {icon}  {check_name}')

In [ ]:
# Inspect the generated Rocq proof (if coqc is available)
if result.rocq_proof:
    print('Generated Rocq proof:')
    print('─' * 60)
    print(result.rocq_proof)
    print('─' * 60)
    print('Verified by coqc:', result.rocq_verified)

---
## Feedback Loop — Layer 9 → Layer 7

When validation fails, `FeedbackController` tightens the VAE sampling constraints and re-generates candidates.

In [ ]:
from src.llm.feedback_controller import FeedbackController

controller = FeedbackController()
constraints = controller.extract_constraints(result)

print('Updated sampling constraints after failed validation:')
for k, v in constraints.items():
    print(f'  {k}: {v}')

# Apply soft constraint to the VAE reparameterize step
controller.update_vae_sampling(vae, constraints)
print('\nVAE sampling updated — re-run Layer 7 to generate constrained candidates.')

---
## Layer 10 — Multi-Fidelity Screening

> **Status: Not yet implemented.**

Planned hierarchical filtering pipeline:

```
Fast ML score  →  Molecular Dynamics  →  Quantum Chemistry  →  Experimental assay
  (~1 ms/mol)       (~1 min/mol)           (~1 hr/mol)          (~1 day/mol)
```

Each stage eliminates ~90% of candidates, so only the top ~0.001% of generated molecules reach wet-lab testing.

In [ ]:
# TODO: implement multi-fidelity screening
#
# Suggested libraries:
#   - OpenMM / GROMACS bindings  → molecular dynamics
#   - PySCF / Psi4               → quantum chemistry (DFT)
#   - AutoDock-GPU               → docking scores as proxy for binding affinity

def multifidelity_screen(smiles_list):
    """Placeholder — returns all candidates unchanged."""
    raise NotImplementedError('Layer 10 is not yet implemented.')

print('Layer 10 placeholder registered.')

---
## End-to-End Pipeline Demo

Runs the full pipeline on an example patient and a small pool of candidate molecules.

In [ ]:
CANDIDATE_SMILES = [
    'CC(=O)Oc1ccccc1C(=O)O',  # Aspirin
    'CC12CCC3C(C1CCC2O)CCC4=CC(=O)CCC34C',  # Testosterone
    'CN1CCC[C@H]1c2cccnc2',    # Nicotine
    'C1=CC=C2C(=C1)C=CC=C2',   # Azulene
    'c1ccc2c(c1)cc1ccc3cccc4ccc2c1c34',  # Pyrene (PAINS alert expected)
]

print(f'Validating {len(CANDIDATE_SMILES)} candidate molecules...\n')
validator = DrugValidator()

results = []
for smi in CANDIDATE_SMILES:
    mol = Chem.MolFromSmiles(smi)
    name = mol.GetPropsAsDict().get('_Name', smi[:20]) if mol else smi[:20]
    try:
        res = validator.validate(smi)
        results.append((smi, res.confidence_score, res.checks))
        status = '✅ PASS' if res.confidence_score >= 0.75 else '⚠  LOW' if res.confidence_score >= 0.5 else '❌ FAIL'
        print(f'{status}  score={res.confidence_score:.2f}  {smi[:35]}')
    except Exception as e:
        print(f'ERROR  {smi[:35]}  ({e})')

# Rank survivors
survivors = [(smi, score) for smi, score, _ in results if score >= 0.75]
survivors.sort(key=lambda x: -x[1])

print(f'\n{len(survivors)}/{len(CANDIDATE_SMILES)} candidates cleared for Layer 10 screening.')

---
## Results & Discussion

### What worked well
- The rule-based scoring in Layers 3–6 is fully deterministic and auditable — every target confidence score can be traced back to a specific API response or formula weight.
- The Rocq formal verification library covers all 118 elements, 12+ drug-likeness filters, and produces machine-checkable proofs — a genuine novelty over existing pipelines.
- The MC-Dropout property predictor gives calibrated uncertainty estimates, flagging low-confidence predictions before they propagate to expensive downstream steps.

### Current limitations & next steps

| Gap | Suggested fix |
|-----|---------------|
| Layer 2 — no ML disease classifier | Fine-tune a small transformer or GBT on TCGA/GEO labelled expression data |
| Layer 7 — hardcoded SMILES pool | Deploy trained VAE decoder; or integrate REINVENT 4 / DiffSBDD for structure-based generation |
| Layer 10 — no docking | Wrap AutoDock-GPU or Vina as a subprocess; feed docking score as an additional reward signal to L7 |
| Fingerprint inversion | Train a separate graph decoder (CGVAE / JTVAE) that maps latent vectors to valid molecular graphs |
| LLM proof reliability | Fine-tune a small code model on `rocq_proof_corpus.jsonl` to reduce retry count |


In [ ]:
# Summary statistics
if results:
    scores = [s for _, s, _ in results]
    print(f'Candidates evaluated : {len(scores)}')
    print(f'Mean confidence score: {np.mean(scores):.3f}')
    print(f'Max confidence score : {max(scores):.3f}')
    print(f'Passing (≥ 0.75)     : {sum(1 for s in scores if s >= 0.75)}')